## Logon and get creds from keyvault

In [1]:
Import-Module az.accounts
Connect-AzAccount |Out-String -width 150 #Login via browser. Add params for other methods as needed 


Account                                         SubscriptionName                  TenantId                             Environment
-------                                         ----------------                  --------                             -----------
subscriptionowner@emsmonitoring.onmicrosoft.com EMS Monitoring Azure Subscription c348c0de-094b-4ffe-bbf1-b533a981f2c4 AzureCloud




The exchange to do a browser logon. 
```
https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize?
scope=https://management.core.windows.net//.default+
           openid
           profile
           offline_access
response_type=code
client_id=1950a258-227b-4e31-a9cf-717495945fc2              <== Well known client ID for Azure PowerShell 
redirect_uri=http://localhost:8400/
claims={"access_token": {"xms_cc":{"values": ["CP1"]  }}}
client-request-id=eb110687-9c3c-4b7e-8d15-607f1f585927
x-client-SKU=MSAL.NetCore                            <= MsalAccessTokenAcquirer ?? 
x-client-Ver=4.23.0.0
x-client-OS=Microsoft+Windows+10.0.18363
prompt=select_account
code_challenge=xxx 
code_challenge_method=S256
state=541f234a-8ac7-4a5e-bff1-fe428eafce23dba10bb3-a3f6-4ddc-8a08-646e19b052ee <== GUID + GUID !? 
sso_nonce=xxx
mscrid=eb110687-9c3c-4b7e-8d15-607f1f585927  <= Same as client -request-id above. 

http://localhost:8400/?                                                         <= Passed as redirect URI 
code=0.Axxx                                                                     <== Code to go fetch
state=541f234a-8ac7-4a5e-bff1-fe428eafce23dba10bb3-a3f6-4ddc-8a08-646e19b052ee  <= Links to request
session_state=2502c53b-bcc3-4a81-83a9-cae9e61b0be4
````

In [1]:
Import-Module Az.KeyVault 
$kvName = "amkeysprod" # OR whatever
$v = Get-Azkeyvault -VaultName $kvName 
$v | Get-AzKeyVaultSecret | Format-Table vaultName,Name, Created  |Out-String # just so we can see that we can in the next step. 


VaultName  Name           Created
---------  ----           -------
amkeysprod ADOAdminName   10/03/2021 11:54:36
amkeysprod ADOAdminPwd    12/01/2021 13:25:00
amkeysprod AppID          15/01/2021 13:15:26
amkeysprod saName         12/01/2021 13:25:03
amkeysprod saPwd          12/01/2021 13:25:05
amkeysprod SPPwd          15/01/2021 13:17:35
amkeysprod SubscriptionID 15/01/2021 13:16:18
amkeysprod TenantId       15/01/2021 12:53:59




In [1]:
$Tenantid =  $v | Get-AzKeyVaultSecret -Name Tenantid -AsPlainText
$AppID    =  $v | Get-AzKeyVaultSecret -Name AppID    -AsPlainText
$SPPwd    =  $v | Get-AzKeyVaultSecret -Name SPPwd   -AsPlainText     
#No we're not going to print them in the notebook 
$appid

60c64680-93ea-4675-8d01-878a426abfad


## AzCopy - try logging on with creds from the vault and accessing blob containers

[Link to the the docs](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10)  

It is preinstalled in Machines running Azure pipelines.  [Download link for local use ](https://aka.ms/downloadazcopy-v10-windows)  When it goes out of date you get a message like this:

`INFO: azcopy.exe: A newer version 10.10.0 is available to download`

[Another useful link I have bookmarked](https://adamtheautomator.com/azure-sas-token/)

In [1]:
del Env:\AZCOPY_CRED_TYPE -ErrorAction SilentlyContinue
if (Test-Path 'C:\Program Files\AzCopy\azcopy.exe') {
    Push-Location 'C:\Program Files\AzCopy'
    $env:AZCOPY_SPA_CLIENT_SECRET = $sppwd 
    .\azcopy.exe login  --service-principal --application-id $appID --tenant-id $tenantID
}

INFO: If you set an environment variable by using the command line, that variable will be readable in your command line history. Consider clearing variables that contain credentials from your command line history.  To keep variables from appearing in your history, you can use a script to prompt the user for their credentials, and to set the environment variable.


INFO: SPN Auth via secret succeeded.


**Our container will fail with this style of login** 

The Service account could logon (and can enumerate containers - if the container is omitted from the URI below we get an error per container)     
*but we can't use that logon to see the contents of a containers*. 

Azcopy has this message *To be authorized to your Azure Storage account, you must assign the **Storage Blob Data Contributor** role to your user account in the context of either the Storage account, parent resource group or parent subscription.* but even granting that role  or doing Azcopy login to interactively login as the subscription owner doesn't give access this way.

In [1]:
.\azcopy.exe  list "https://saamdocuments.blob.core.windows.net/amdev-moj"  | out-string
.\azcopy.exe  logout
Pop-location
del Env:\AZCOPY_SPA_CLIENT_SECRET

INFO: Authenticating to destination using Azure AD

failed to traverse container: cannot list files due to reason -> github.com/Azure/azure-storage-blob-go/azblob.newStorageError, /home/vsts/go/pkg/mod/github.com/!azure/azure-storage-blob-go@v0.10.1-0.20210407023846-16cf969ec1c3/azblob/zc_storage_error.go:42
===== RESPONSE ERROR (ServiceCode=AuthorizationPermissionMismatch) =====
Description=This request is not authorized to perform this operation using this permission.
RequestId:e22dca66-a01e-005d-6651-386f7f000000
Time:2021-04-23T15:03:14.0297728Z, Details: 
   Code: AuthorizationPermissionMismatch
   GET https://saamdocuments.blob.core.windows.net/amdev-moj?comp=list&delimiter=%2F&include=metadata&restype=container&timeout=901
   Authorization: REDACTED
   User-Agent: [AzCopy/10.10.0 Azure-Storage/0.13 (go1.15; Windows_NT)]
   X-Ms-Client-Request-Id: [4b83eb04-5f4a-46fa-5f77-7512b6925177]
   X-Ms-Version: [2019-12-12]
   --------------------------------------------------------------

INFO: Logout succeeded.


## PowerShell access to blob containers and contents. 

In [1]:
ipmo az.storage
Get-AzStorageAccount |Out-String -Width 150


StorageAccountName  ResourceGroupName              PrimaryLocation SkuName      Kind      AccessTier CreationTime        ProvisioningState EnableHttps
                                                                                                                                           TrafficOnly
------------------  -----------------              --------------- -------      ----      ---------- ------------        ----------------- -----------
csb1003200108f87531 cloud-shell-storage-westeurope westeurope      Standard_LRS StorageV2 Hot        26/01/2021 13:59:17 Succeeded         True       
csb1003200108f8758e cloud-shell-storage-westeurope westeurope      Standard_LRS StorageV2 Hot        12/01/2021 11:51:15 Succeeded         True       
csb100320010992082d cloud-shell-storage-westeurope westeurope      Standard_LRS StorageV2 Hot        06/01/2021 15:44:09 Succeeded         True       
saamdiaglogging     rg-amProd-moj                  uksouth         Standard_LRS StorageV2 Hot

The next cell sets up a context. 

We need to be logged on to get the storage account key (and I don't want to save it here!)      
but **with a context we don't need to be logged into to Azure**.  

So the `Get-AzStorageBlob -context` isn't dependent on the logon. and for a pipeline we'd have     
storage-account-key => keyvault => variable group => environment variable  

In [1]:
$sa      = Get-AzStorageAccount | where StorageAccountName -eq "saamdocuments"
$context = New-AzStorageContext -StorageAccountName $sa.StorageAccountName -StorageAccountKey  ($sa | Get-AzStorageAccountKey | where keyName -eq "key1").value 
Get-AzStorageBlob -Container "amdev-moj" -Context $Context | ft name, length, contenttype,LastModified -a | out-string -Width 200


Name                                                                    Length ContentType                                                             LastModified
----                                                                    ------ -----------                                                             ------------
AAMR0205/ORD0093/Enforcement Documents/Doc1.rtf                          42029 application/msword                                                      16/04/2021 16:35:25 +00:00
AAMR0205/ORD0093/Enforcement Documents/ENF (1) (1) (1) (2) (1) (1).docx  11820 application/vnd.openxmlformats-officedocument.wordprocessingml.document 26/03/2021 08:39:14 +00:00




`Get-AzStorageBlobContent` Copies a blob to the local machine - again with context we don't need to be logged on. Local folder creation is automatic.  

In [1]:
Get-AzStorageBlob -Container "amdev-moj" -Context $Context | Get-AzStorageBlobContent -Destination \temp -force

   AccountName: saamdocuments, ContainerName: amdev-moj


Name                 BlobType  Length          ContentType                    LastModified


----                 --------  ------          -----------                    ------------        


AAMR0205/ORD0093/En… BlockBlob 42029           application/msword             2021-04-16 16:35:25Z


AAMR0205/ORD0093/En… BlockBlob 11820           application/vnd.openxmlformat… 2021-03-26 08:39:14Z


In [1]:
dir 'C:\temp\AAMR0205\ORD0093\Enforcement Documents' | Out-String


    Directory: C:\temp\AAMR0205\ORD0093\Enforcement Documents

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---          23/04/2021    16:05          42029 Doc1.rtf
-a---          23/04/2021    16:05          11820 ENF (1) (1) (1) (2) (1) (1).docx




## Creating a Sas token for blob containers and using it with AzCopy
**We have already created a storage context** (so we don't need to logon). 

Create a SAS-Token which is valid for one container, from the local ip, for 10 Minutes over HTTPS.

In [1]:
$myIp = (Invoke-WebRequest http://api.ipify.org/).content
$myip

81.109.118.32


In [1]:
(New-AzStorageContainerSASToken -Context $context -name 'amdev-moj' -ExpiryTime ([datetime]::now.AddMinutes(10)) `
   -FullUri -IPAddressOrRange $myip -Permission 'rwdl' -Protocol  httpsOnly -OutVariable sastoken) -replace '\?|&',"`n"

https://saamdocuments.blob.core.windows.net/amdev-moj
sv=2019-07-07
sr=c
sig=BWJURetr1hjocmPFCpHjJlKbrAcjfLvIlH4yr6aZOWw%3D
spr=https
sip=81.109.118.32
se=2021-04-23T15%3A15%3A47Z
sp=rwdl


Token breaks down like this:

```
https://<<Storage Account Name>>.blob.core.windows.net/<<Storage Account Name>>
sv=2019-07-07                <<Sevice version>>
sr=c                         <<Storage resource  c=container , b=blob, s=share etc. >>
spr=https                    <<Protocol  only one option for this "httpsonly"  >>
sip=81.109.118.32            <<IP>>
se=2021-04-15T09%3A11%3A47Z  <<Expiry time   st= would be start time we didn't request one>>
sp=rwdl                      <<Permissions read , write, delete, list >>
sig=Eq2yJoz...               <<Signature ties the above to a key>>
```

In [1]:
del Env:\AZCOPY_SPA_CLIENT_SECRET -ErrorAction SilentlyContinue
if (Test-Path 'C:\Program Files\AzCopy\azcopy.exe') {
    Push-Location 'C:\Program Files\AzCopy'
    $env:AZCOPY_CRED_TYPE = "Anonymous";
    .\azcopy.exe  copy $sasToken C:\temp\azcopy\ --recursive --overwrite true
}

INFO: Scanning...


INFO: Any empty folders will not be processed, because source and/or destination doesn't have full folder support


Job f4ec3c44-9549-7b48-61b9-be9a25bedb76 has started


Log file is located at: C:\Users\P10506111\.azcopy\f4ec3c44-9549-7b48-61b9-be9a25bedb76.log


0.0 %, 0 Done, 0 Failed, 2 Pending, 0 Skipped, 2 Total, 


...

Number of Transfers Completed: 2


Number of Transfers Failed: 0


Number of Transfers Skipped: 0


TotalBytesTransferred: 53849


Final Job Status: Completed


In [1]:
type -raw C:\Users\P10506111\.azcopy\f4ec3c44-9549-7b48-61b9-be9a25bedb76.log

2021/04/23 15:06:45 AzcopyVersion  10.10.0
2021/04/23 15:06:45 OS-Environment  windows
2021/04/23 15:06:45 OS-Architecture  amd64
2021/04/23 15:06:45 Log times are in UTC. Local time is 23 Apr 2021 16:06:45
2021/04/23 15:06:45 Job-Command copy https://saamdocuments.blob.core.windows.net/amdev-moj?se=2021-04-23t15%3A15%3A47z&sig=-REDACTED-&sip=81.109.118.32&sp=rwdl&spr=https&sr=c&sv=2019-07-07 C:\temp\azcopy\ --recursive --overwrite true 
2021/04/23 15:06:45 Number of CPUs: 8
2021/04/23 15:06:45 Max file buffer RAM 4.000 GB
2021/04/23 15:06:45 Max concurrent network operations: 128 (Based on number of CPUs. Set AZCOPY_CONCURRENCY_VALUE environment variable to override)
2021/04/23 15:06:45 Check CPU usage when dynamically tuning concurrency: true (Based on hard-coded default. Set AZCOPY_TUNE_TO_CPU environment variable to true or false override)
2021/04/23 15:06:45 Max concurrent transfer initiation routines: 64 (Based on hard-coded default. Set AZCOPY_CONCURRENT_FILES environment variab

Copy in the other direction

In [1]:
.\azcopy.exe  copy C:\temp\source $sasToken  --recursive --overwrite true

INFO: Scanning...


INFO: Any empty folders will not be processed, because source and/or destination doesn't have full folder support


Job b637387e-21e2-744f-4b0c-a8717d2f27fa has started


Log file is located at: C:\Users\P10506111\.azcopy\b637387e-21e2-744f-4b0c-a8717d2f27fa.log


0.0 %, 0 Done, 0 Failed, 13 Pending, 0 Skipped, 13 Total, 


...

Number of Transfers Completed: 13


Number of Transfers Failed: 0


Number of Transfers Skipped: 0


TotalBytesTransferred: 54726


Final Job Status: Completed


**Time passes ....**

The token expires

In [1]:
if (Test-Path 'C:\Program Files\AzCopy\azcopy.exe') {
    Push-Location 'C:\Program Files\AzCopy'
    $env:AZCOPY_CRED_TYPE = "Anonymous";.\azcopy.exe  list $sasToken
}    

failed to traverse container: cannot list files due to reason -> github.com/Azure/azure-storage-blob-go/azblob.newStorageError, /home/vsts/go/pkg/mod/github.com/!azure/azure-storage-blob-go@v0.10.1-0.20210407023846-16cf969ec1c3/azblob/zc_storage_error.go:42


===== RESPONSE ERROR (ServiceCode=AuthenticationFailed) =====


Description=Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.


RequestId:3b51bff8-301e-005c-6853-38447d000000


Time:2021-04-23T15:17:20.9145739Z, Details: 


...

   Content-Type: [application/xml]


   Date: [Fri, 23 Apr 2021 15:17:20 GMT]


   Server: [Microsoft-HTTPAPI/2.0]


   X-Ms-Error-Code: [AuthenticationFailed]


   X-Ms-Request-Id: [3b51bff8-301e-005c-6853-38447d000000]


In [1]:
del  Env:\AZCOPY_CRED_TYPE -ErrorAction SilentlyContinue
Pop-Location

## File share instead of blob containers. 

Change of storage account. **Create new context**


In [1]:
$sa2 = Get-AzStorageAccount | where StorageAccountName -eq CSB1003200108f87531 
$sa2 | Get-AzStorageshare | Get-AzStoragefile | where {$_.CloudFileDirectory } | Get-AzStoragefile | Out-String


   Directory: https://csb1003200108f87531.file.core.windows.net/cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531/.cloudconsole

Type                Length Name
----                ------ ----
File            5368709120 acc_subscription.img

   Directory: https://csb1003200108f87531.file.core.windows.net/cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531/AAMR0205

Type                Length Name
----                ------ ----
Directory                1 ORD0093

   Directory: https://csb1003200108f87531.file.core.windows.net/cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531/source

Type                Length Name
----                ------ ----
Directory                1 .docs
Directory                1 classes
Directory                1 private
Directory                1 public
File                   683 VSTeam.psm1




In [1]:
$sa2 | Get-AzStorageshare | Get-AzStoragefile | where {-not $_.CloudFileDirectory } | Get-AzStorageFileContent -Destination c:\temp -PassThru -Force | out-string


   Directory: https://csb1003200108f87531.file.core.windows.net/cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531

Type                Length Name
----                ------ ----
File                152149 auditrecords.json
File                  4487 dbusers.csv
File                   951 foo.csv
File                   330 groups.csv
File                  2816 Import-Groups.ps1
File                   683 VSTeam.psm1




In [1]:
$context2 = New-AzStorageContext -StorageAccountName $sa2.StorageAccountName -StorageAccountKey  ($sa2 | Get-AzStorageAccountKey | where keyName -eq "key1").value 
$destshare = Get-AzStorageShare  -Context $context2 -name 'cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531'
cd C:\temp\source
$CurrentFolder = (Get-Item .).FullName
Get-ChildItem -Recurse | Where-Object { $_.GetType().Name -eq "FileInfo"} | ForEach-Object {
        $path=$_.FullName.Substring($Currentfolder.Length+1).Replace("\","/")
        Set-AzStorageFileContent -ShareName 'cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531' -Source $_.FullName -Path $path -Force -Context $Context2 2> $env:temp\err.txt
    }
write-host "Any item where the path doesn't exist will give an error that's all of these the - last one was " 
Get-Content $env:temp\err.txt -raw
Remove-Item $env:temp\err.txt -ErrorAction SilentlyContinue 

Any item where the path doesn't exist will give an error that's all of these the - last one was 



Set-AzStorageFileContent: 
Line |
   7 |          Set-AzStorageFileContent -ShareName 'cs-subscriptionowner-ems …
     |          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The specified parent path does not exist. HTTP Status Code: 404 - HTTP Error Message: The specified parent path does not exist.
ErrorCode: ParentNotFound
ErrorMessage: The specified parent path does not exist.
RequestId:30752c89-201a-0057-4553-38ef64000000
Time:2021-04-23T15:16:11.6415964Z



In [1]:
$sa2 = Get-AzStorageAccount | where StorageAccountName -eq CSB1003200108f87531 
$context2 = New-AzStorageContext -StorageAccountName $sa2.StorageAccountName -StorageAccountKey  ($sa2 | Get-AzStorageAccountKey | where keyName -eq "key1").value 

$sasToken2 = New-AzStorageShareSASToken -Context $context2 -ExpiryTime ([datetime]::now.AddMinutes(5)) `
                -fulluri -IPAddressOrRange $myIp -Permission "rwdl" -protocol "httpsOnly" `
                -ShareName 'cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531' 

del Env:\AZCOPY_SPA_CLIENT_SECRET -ErrorAction SilentlyContinue
if (Test-Path 'C:\Program Files\AzCopy\azcopy.exe') {
    Push-Location 'C:\Program Files\AzCopy'    
    $env:AZCOPY_CRED_TYPE = "Anonymous";
    .\azcopy.exe list $sasToken2  | Out-String
}

INFO: /;  Content Length: 0.00 B
INFO: dbusers.csv;  Content Length: 4.38 KiB
INFO: groups.csv;  Content Length: 330.00 B
INFO: VSTeam.psm1;  Content Length: 683.00 B
INFO: source/;  Content Length: 0.00 B
INFO: AAMR0205/;  Content Length: 0.00 B
INFO: Import-Groups.ps1;  Content Length: 2.75 KiB
INFO: .cloudconsole/;  Content Length: 0.00 B
INFO: auditrecords.json;  Content Length: 148.58 KiB
INFO: foo.csv;  Content Length: 951.00 B
INFO: .cloudconsole/acc_subscription.img;  Content Length: 5.00 GiB
INFO: source/private/;  Content Length: 0.00 B
INFO: source/.docs/;  Content Length: 0.00 B
INFO: source/public/;  Content Length: 0.00 B
INFO: source/VSTeam.psm1;  Content Length: 683.00 B
INFO: source/classes/;  Content Length: 0.00 B
INFO: AAMR0205/ORD0093/;  Content Length: 0.00 B
INFO: source/.docs/Get-VSTeamProcess.md;  Content Length: 1.83 KiB
INFO: source/.docs/Get-VSTeamWorkItemType.md;  Content Length: 2.59 KiB
INFO: source/.docs/synopsis/;  Content Length: 0.00 B
INFO: source/pr

In [1]:
.\azcopy.exe copy $($sasToken2 -replace "\?", "/*?") $env:temp

INFO: Scanning...


INFO: Any empty folders will not be processed, because --recursive was not specified


Job 0b645712-b3f7-9442-7bb3-11241bcc114b has started


Log file is located at: C:\Users\P10506111\.azcopy\0b645712-b3f7-9442-7bb3-11241bcc114b.log


0.0 %, 0 Done, 0 Failed, 6 Pending, 0 Skipped, 6 Total, 


...

Number of Transfers Completed: 6


Number of Transfers Failed: 0


Number of Transfers Skipped: 0


TotalBytesTransferred: 161416


Final Job Status: Completed


Copy in the other direction

In [1]:
.\azcopy.exe  copy C:\temp\source $sasToken2  --recursive --overwrite true

INFO: Scanning...


INFO: Any empty folders will be processed, because source and destination both support folders


Job a3e84daf-d580-a644-7aef-5e4652380635 has started


Log file is located at: C:\Users\P10506111\.azcopy\a3e84daf-d580-a644-7aef-5e4652380635.log


0.0 %, 0 Done, 0 Failed, 19 Pending, 0 Skipped, 19 Total, 


...

Number of Transfers Completed: 19


Number of Transfers Failed: 0


Number of Transfers Skipped: 0


TotalBytesTransferred: 54726


Final Job Status: Completed


## Back to back without local copy

This won't run from local IP. Keep the token windows short, don't get write access on the read side etc. 

In [1]:
$sastoken = New-AzStorageContainerSASToken -FullUri  -Context $context  -name               'amdev-moj' `
                                           -ExpiryTime ([datetime]::now.AddMinutes(2))    -Permission 'rl' -Protocol  httpsOnly 

$sasToken2 = New-AzStorageShareSASToken    -FullUri  -Context $context2 -ShareName        'cs-subscriptionowner-emsmonitoring-onmicrosoft-1003200108f87531' `
                                           -ExpiryTime ([datetime]::now.AddMinutes(2))    -Permission 'rwdl' -Protocol httpsOnly 

.\azcopy.exe  copy $sastoken  $sasToken2  --recursive 

INFO: Scanning...


INFO: Failed to create one or more destination container(s). Your transfers may still succeed if the container already exists.


INFO: Any empty folders will not be processed, because source and/or destination doesn't have full folder support


Job 8e501daf-a85b-0c47-66d1-870f2f689d51 has started


Log file is located at: C:\Users\P10506111\.azcopy\8e501daf-a85b-0c47-66d1-870f2f689d51.log


...

Number of Transfers Completed: 15


Number of Transfers Failed: 0


Number of Transfers Skipped: 0


TotalBytesTransferred: 108575


Final Job Status: Completed
